<a href="https://colab.research.google.com/github/CMccormick0003/home_sales_big_data/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
SPARK_VERSION = 'spark-3.4.1'

URL = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [2]:
import os
os.environ['SPARK_VERSION'] = SPARK_VERSION

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{SPARK_VERSION}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

# Import packages
from pyspark.sql import SparkSession
import time

from pyspark import SparkFiles

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,360 kB]
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,299 kB]
Fetched 4,999 kB i

In [3]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.


spark.sparkContext.addFile(URL)
home_sales_df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.


home_sales_df.createOrReplaceTempView('home_sales')

In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?


spark.sql("select round(avg(price),2) from home_sales where bedrooms = 4").show()

+--------------------+
|round(avg(price), 2)|
+--------------------+
|           299661.01|
+--------------------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("select date_built, round(avg(price),2) from home_sales where bathrooms = 3 and bedrooms = 3 group by date_built order by date_built").show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           292859.62|
|      2011|           291117.47|
|      2012|           293683.19|
|      2013|           295962.27|
|      2014|           290852.27|
|      2015|            288770.3|
|      2016|           290555.07|
|      2017|           292676.79|
+----------+--------------------+



In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("select date_built, round(avg(price),2) from home_sales where bathrooms = 3 and bedrooms = 3 and floors = 2 and sqft_living >= 2000 group by date_built order by date_built").show()


+----------+--------------------+
|date_built|round(avg(price), 2)|
+----------+--------------------+
|      2010|           285010.22|
|      2011|           276553.81|
|      2012|           307539.97|
|      2013|           303676.79|
|      2014|           298264.72|
|      2015|           297609.97|
|      2016|            293965.1|
|      2017|           280317.58|
+----------+--------------------+



In [9]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("select round(avg(price),2), avg(view) from home_sales where price >= 350000").show()


print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+---------+
|round(avg(price), 2)|avg(view)|
+--------------------+---------+
|           473796.26|   32.264|
+--------------------+---------+

--- 0.7700293064117432 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")


DataFrame[]

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [12]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("select round(avg(price),2), avg(view) from home_sales where price >= 350000").show()

print("--- %s seconds ---" % (time.time() - start_time))


+--------------------+---------+
|round(avg(price), 2)|avg(view)|
+--------------------+---------+
|           473796.26|   32.264|
+--------------------+---------+

--- 0.7861204147338867 seconds ---


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data.
# The '_SUCCESS' file in the left folder icon is created when Spark creates a Parquet folder. The part-* binary files store compressed data as columns.
home_sales_df.write.parquet('parquet_home_sales', mode='overwrite')


In [14]:
# 11. Read the parquet formatted data. This has all the same methods as a row-based df.
p_home_sales_df=spark.read.parquet('parquet_home_sales')

In [15]:
# 12. Create a temporary table for the parquet data.
p_home_sales_df.createOrReplaceTempView('p_home_sales')


In [16]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
spark.sql("select round(avg(price),2), avg(view) from p_home_sales where price >= 350000").show()
print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+---------+
|round(avg(price), 2)|avg(view)|
+--------------------+---------+
|           473796.26|   32.264|
+--------------------+---------+

--- 0.5956718921661377 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [18]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")

False